# ConvNet optimization with GA

caveat: Fa schifo, l'ho fatto girare un'ora su kaggle e raggiunge al massimo 50 % di accuracy ed  è moolto lento. L'ho fatto girare solo su un subset di MNIST perchè se no non finiva più

### Setup

In [1]:
! pip install pygad
! pip install torchsummary

Purtroppo pygad usa di default cuda e nella parte di ottimizazione il programma si blocca perchè i weights sono gpu e gli input su cpu. Ho provato a spostare tutto su gpu ma mi crasha il kernel. Per disabilitare cuda:

In [ ]:
!export CUDA_VISIBLE_DEVICES=""

In [2]:
import torch
import pygad.torchga
import pygad
import numpy
from torchsummary import summary

/home/alexserra98/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Helper functions

In [3]:
def fitness_func(solution, sol_idx):
    global data_inputs, data_outputs, torch_ga, model, loss_function

    predictions = pygad.torchga.predict(model=model, 
                                        solution=solution, 
                                        data=data_inputs)

    solution_fitness = 1.0 / (loss_function(predictions, data_outputs).detach().numpy() + 0.00000001)

    return solution_fitness

def callback_generation(ga_instance):
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))


### Model

In [5]:
# Build the PyTorch model.
input_layer = torch.nn.Conv2d(in_channels=3, out_channels=5, kernel_size=7)
relu_layer1 = torch.nn.ReLU()
max_pool1 = torch.nn.MaxPool2d(kernel_size=5, stride=5)

conv_layer2 = torch.nn.Conv2d(in_channels=5, out_channels=3, kernel_size=3)
relu_layer2 = torch.nn.ReLU()

flatten_layer1 = torch.nn.Flatten()
# The value 768 is pre-computed by tracing the sizes of the layers' outputs.
dense_layer1 = torch.nn.Linear(in_features=12, out_features=12)
relu_layer3 = torch.nn.ReLU()

dense_layer2 = torch.nn.Linear(in_features=12, out_features=10)
output_layer = torch.nn.Softmax(1)

mylist = [input_layer,
         relu_layer1,
         max_pool1,
         conv_layer2,
         relu_layer2,
         flatten_layer1,
         dense_layer1,
         relu_layer3,
         dense_layer2,
         output_layer]

model = torch.nn.Sequential(*mylist)

In [4]:
summary(model,(3,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 5, 22, 22]           740
├─ReLU: 1-2                              [-1, 5, 22, 22]           --
├─MaxPool2d: 1-3                         [-1, 5, 4, 4]             --
├─Conv2d: 1-4                            [-1, 3, 2, 2]             138
├─ReLU: 1-5                              [-1, 3, 2, 2]             --
├─Flatten: 1-6                           [-1, 12]                  --
├─Linear: 1-7                            [-1, 12]                  156
├─ReLU: 1-8                              [-1, 12]                  --
├─Linear: 1-9                            [-1, 10]                  130
├─Softmax: 1-10                          [-1, 10]                  --
Total params: 1,164
Trainable params: 1,164
Non-trainable params: 0
Total mult-adds (M): 0.36
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.03


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 5, 22, 22]           740
├─ReLU: 1-2                              [-1, 5, 22, 22]           --
├─MaxPool2d: 1-3                         [-1, 5, 4, 4]             --
├─Conv2d: 1-4                            [-1, 3, 2, 2]             138
├─ReLU: 1-5                              [-1, 3, 2, 2]             --
├─Flatten: 1-6                           [-1, 12]                  --
├─Linear: 1-7                            [-1, 12]                  156
├─ReLU: 1-8                              [-1, 12]                  --
├─Linear: 1-9                            [-1, 10]                  130
├─Softmax: 1-10                          [-1, 10]                  --
Total params: 1,164
Trainable params: 1,164
Non-trainable params: 0
Total mult-adds (M): 0.36
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.03

### Data

Il dataset è MNIST ma convertito in .npy, qua trovate dove scaricarlo
url: https://www.kaggle.com/datasets/sivasankaru/mnist-npy-file-dataset/versions/1?resource=download

In [37]:
data_inputs = torch.from_numpy(numpy.load("../input/mnist-npy-file-dataset/train_images.npy",allow_pickle=True).reshape((60000,28,28))).float()
data_outputs = torch.from_numpy(numpy.load("../input/mnist-npy-file-dataset/train_labels.npy", allow_pickle=True)).long()
data_inputs = torch.stack((data_inputs,torch.clone(data_inputs),torch.clone(data_inputs)), dim=1)
data_inputs=data_inputs[0:10000]
data_outputs=data_outputs[0:10000]
data_outputs=data_outputs.reshape((10000))


### GA optimization

In [5]:
# Create an instance of the pygad.torchga.TorchGA class to build the initial population.
torch_ga = pygad.torchga.TorchGA(model=model,
                                 num_solutions=10)

loss_function = torch.nn.CrossEntropyLoss()

In [40]:
# Prepare the PyGAD parameters. Check the documentation for more information: https://pygad.readthedocs.io/en/latest/README_pygad_ReadTheDocs.html#pygad-ga-class
num_generations = 200 # Number of generations.
num_parents_mating = 5 # Number of solutions to be selected as parents in the mating pool.
initial_population = torch_ga.population_weights # Initial population of network weights.

# Create an instance of the pygad.GA class
ga_instance = pygad.GA(num_generations=num_generations, 
                       num_parents_mating=num_parents_mating, 
                       initial_population=initial_population,
                       fitness_func=fitness_func,
                       on_generation=callback_generation)

# Start the genetic algorithm evolution.
ga_instance.run()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

### Eval

In [1]:
# After the generations complete, some plots are showed that summarize how the outputs/fitness values evolve over generations.
ga_instance.plot_fitness(title="PyGAD & PyTorch - Iteration vs. Fitness", linewidth=4)

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

predictions = pygad.torchga.predict(model=model, 
                                    solution=solution, 
                                    data=test_inputs)
# print("Predictions : \n", predictions)

# Calculate the crossentropy for the trained model.
print("Crossentropy : ", loss_function(predictions, test_outputs).detach().numpy())

# Calculate the classification accuracy for the trained model.
accuracy = torch.true_divide(torch.sum(torch.max(predictions, axis=1).indices == test_outputs), len(test_outputs))
print("Accuracy : ", accuracy.detach().numpy())

/home/alexserra98/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

: 